In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from scipy.io import loadmat
import scipy
#import hdf5storage
import csv
from tqdm import trange

import glob

Autosaving every 180 seconds


In [2]:
###########################################################
###### CONVERT DATA TO JAABA trx.mat FORMAT ###############
###########################################################

def convert_npy_to_mat_for_jaaba(fname, 
                                 sexes,
                                 start,
                                 end,
                                dtype,
                                animal_ids,
                                 make_video
                                ):
    
    from scipy.io import savemat
    
    data = np.load(fname)
    
    if end is None:
        end = data.shape[1]
        
    trx_array=[]
    for k in animal_ids:
        x = np.array(data[k,start:end,0], dtype) # x-coordinate of the animal in pixels (1 x nframes).
        y = np.array(data[k,start:end,1], dtype) # y-coordinate of the animal in pixels (1 x nframes).
        
        print (' # of data points: ', x.shape)
        # rotations and body shrinking/stretching
        theta = x*0  # TO BE RECOMPUTED FROM FULL FEATURES:  Orientation of the animal (head) (1 x nframes); 
        a = x*0+100  # TO BE RECOMPUTED FROM FULL FEATURES:  1/4 of the major-axis length in pixels (1 x nframes).
        b = x*0+50  # TO BE RECOMPUTED FROM FULL FEATURES:  1/4 of the minor-axis length in pixels (1 x nframes).

        # meta data for the video and animal
        nframes = np.array(end-start,dtype)    # Number of frames in the trajectory of the current animal (scalar).
        firstframe = np.array(1,dtype)             # First frame of the animal's trajectory (scalar). USE MATLAB INDEXES
        endframe = np.array(nframes,dtype)   # Last frame of the animal's trajectory (scalar).
        off = 0                    # Offset for computing index into x, y, etc. Always equal to 1 - firstframe (scalar).
        id_ = k+1                  # Identity number of the trajectory (scalar). USE MATLAB VALUES

        # convert from pixels to distances
        pixels_per_mm = 2                       # number of pixels per mm # to figure out
        x_mm = x//pixels_per_mm                   # x-coordinate of the animal in mm (1 x nframes).
        y_mm = y//pixels_per_mm                   # y-coordinate of the animal in mm (1 x nframes).

        theta_mm = theta                     # Orientation of the animal in real coordinates. This is often the same as theta, if no transformation other than translation and scaling is performed between pixels and real coordinates (1 x nframes).

        a_mm = a//pixels_per_mm  # 1/4 of the major-axis length in mm (1 x nframes).
        b_mm = b//pixels_per_mm  # 1/4 of the major-axis length in mm (1 x nframes).

        sex = sexes[k]  # Sex of the animal. Can be just one value ('M' or 'F' or '?') or a cell array of 'M' and 'F' giving the sex for each frame. The size of the cell array should be nframes.

        # frame-rate based info
        dt = np.ones(int(nframes)-1, dtype)   # Difference in timestamps of the current frame and next frame, in seconds (1 x nframes-1).
        fps = 25                               # Average frames-per-second (scalar).

        days_per_timestamp = 1./(24*60*60*fps)
        timestamps = np.array(np.arange(nframes)*days_per_timestamp,
                             dtype) # Timestamp of each frame (optional), in days (1 x nframes). 

        
        trx ={
         'x': x,
         'y':y,
         'theta': theta,
         'a': a,
         'b': b,
         'nframes':nframes,
         'firstframe':firstframe,
         'endframe':endframe,
         'off':off,
         'id':id_,
         'x_mm':x_mm,
         'y_mm':y_mm,
         'theta_mm':theta_mm,
         'a_mm':a_mm,
         'b_mm':b_mm,
         'sex':sex,
         'dt':dt,
         'fps':fps,
         'timestamps': timestamps
        }
        
        trx_array.append(trx)

    trx_dict = {'a':trx_array[0],
                'b':trx_array[1],
                'c':trx_array[2],
                'd':trx_array[3]               
               }
        
    
#     def split_movie(video_name, start, end):
#         from tqdm import trange

#         fname_out = video_name[:-4]+"_"+str(start)+"_"+str(end)+".mp4"
#         if os.path.exists(fname_out):
#             return

#         #video_name = root_dir + '/' + os.path.split(root_dir)[1]+'.avi'
#         original_vid = cv2.VideoCapture(video_name)

#         # video sizes
#         size_vid = np.array([1280,1024])
#         original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

#         # Initialize video out
#         fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
#         video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0],size_vid[1]), True)

#         for n in trange(start,end, 1):
#             ret, frame = original_vid.read()
#             video_out.write(frame)
#         video_out.release()
#         original_vid.release()

#     if make_video:
#         video_name = os.path.split(fname)[0]+"/movie.avi"

#         # 
#         split_movie(video_name,start,end)
    
    
    savemat(fname[:-4]+'_trx.mat', 
            {
             'trx':trx_dict
            }
           )
    
    
    #
    print ("*** DONE CONVERSION ***")
    
    

In [7]:

fnames = glob.glob('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/post_processing/centres/'+
                   '*.npy')

fname = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed_spine_transposed.npy'
    
sexes = ['M','M','M','M']
sexes = np.ones(4)
fps = 25
start = int(0*60*fps)
end = int(1*60*fps)

dtype = 'double'
animal_ids = np.arange(4)
make_video = False


convert_npy_to_mat_for_jaaba(fname, 
                             sexes,
                             start,
                             end,
                             dtype,
                             animal_ids,
                             make_video)

    

 # of data points:  (1500,)
 # of data points:  (1500,)
 # of data points:  (1500,)
 # of data points:  (1500,)
*** DONE CONVERSION ***


In [6]:
data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed_spine.npy').transpose(1,0,2)
print (data.shape)

np.save('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed_spine_transposed.npy',
       data)

(4, 89989, 2)


In [13]:
from tqdm import trange
clrs = ['red','blue','green','cyan']

dirs = np.sort(glob.glob('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/trx_files/'+
                "*"))

imgs=[]
imgs1 = []
for dir_ in dirs:
    fname_mat = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/trx_files/2020-3-16_01_54_23_358257_compressed.avi.predictions.analysis_traces_reassembled_centres/scores_run.mat'
    data =loadmat(fname_mat)
    
    # get length
    temp = data['allScores'][0][0][0][0][0].squeeze()
    length = temp.shape[0]

    # get starts/ends
    t0s = data['allScores'][0][0][5].squeeze()
    #print (t0s.shape)
    t1s = data['allScores'][0][0][6].squeeze()
    
    img = np.zeros((4,length),'float32')
    img1 = np.zeros((4,length),'float32')
    for k in trange(t0s.shape[0]):
    #for k in [0]:
        starts = t0s[k].squeeze()
        ends = t1s[k].squeeze()
        #print (k,starts.shape)
        for p in range(starts.shape[0]):
            img[k,starts[p]:ends[p]]+=k+1
            img1[k,starts[p]:ends[p]]+=1
            
    idx = np.where(img==0)
    img[idx]=np.nan
    imgs.append(img)
    imgs1.append(img1)
    
imgs=np.hstack(imgs)
imgs1=np.hstack(imgs1)


100%|██████████| 4/4 [00:00<00:00, 15.94it/s]


In [18]:
matplotlib.use('Agg')

fig = plt.figure(figsize=(100,5))
ax=plt.subplot(2,1,1)


plt.xticks([])
print (imgs.shape)
names = ['female','male','pup1','pup2']
clrs = ['red','blue','magenta','green']
plt.yticks(np.arange(4),names)
from matplotlib import colors
cmap = colors.ListedColormap(clrs)

plt.imshow(imgs,aspect='auto',interpolation='none',
          cmap=cmap)

ax=plt.subplot(2,1,2)
sums = np.nansum(imgs1, axis= 0)
print (sums.shape)

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    #y_smooth = np.convolve(y, box, mode='same')
    y_smooth = np.convolve(y,np.ones(box_pts,dtype=int),'same')
    
    return y_smooth

window = 60
t = np.arange(imgs.shape[1])/25./window
img2 = np.zeros((4,imgs.shape[1]))
for k in range(4):
    temp = imgs1[k]
    temp = smooth(temp,25*window)    
    #img2[k]=temp
    #temp = smooth(temp,25)    
    
    plt.plot(t,temp,clrs[k])

#plt.imshow(img2,aspect='auto',interpolation='none')#    cmap=cmap)    
plt.xticks(np.arange(0,600,10))
plt.xlim(t[0],t[-1])
plt.xlabel("Time (minutes)")
plt.suptitle("Run behaviors for all animals ("+str(window)+ "sec smoothing window)", fontsize=20)
plt.savefig('/home/cat/fig.png')
#plt.show()



(4, 899880)
(899880,)


In [100]:
temp = data['allScores'][0][0][0][0][k].squeeze()
print(temp.shape)


(89988,)


In [63]:
t0s = data['allScores'][0][0][5].squeeze()
print (t0s.shape)
t1s = data['allScores'][0][0][6].squeeze()

(4,)


In [167]:
fname_mat = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/jaaba/trx_all_21300_21500.mat'
data = loadmat(fname_mat)

In [168]:
print (data['trx'].shape)
print (data['trx'][0][0][1].shape)

data['trx'][0][0][1][0][0] = 44
data['trx'][0][0][1][0] = np.array(data['trx'][0][0][1][0],dtype)

print (data['trx'][0][0][1][0][0])

(1, 4)
(1, 200)
44.0


In [ ]:
  
    
from numpy.core.records import fromarrays
from scipy.io import savemat

names = [
 'x',
 'y',
 'theta',
 'a',
 'b',
 'nframes',
 'firstframe',
 'endframe',
 'off',
 'id',
 'x_mm',
 'y_mm',
 'theta_mm',
 'a_mm',
 'b_mm',
 'sex',
 'dt',
 'fps',
 'timestamps']

data = [
 x,
 y,
 theta,
 a,
 b,
 nframes,
 firstframe,
 endframe,
 off,
 id_,
 x_mm,
 y_mm,
 theta_mm,
 a_mm,
 b_mm,
 sex,
 dt,
 fps,
 timestamps
]


            
#myrec = fromarrays([[1, 10], [2, 20]], names=['field1', 'field2'])
#savemat('p.mat', {'myrec': myrec})

myrec = fromarrays(data, 
                   names=names)
#savemat('p.mat', {'myrec': myrec})


savemat("/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/jaaba/trx.mat", 
        {'myrec':myrec})
    

In [11]:
fnames = glob.glob('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/post_processing/*.npy')

for fname in fnames:
    data = np.load(fname)
    
    data2 = []
    for f in range(0,56,14):
        temp = data[:,f:f+14]
        temp = np.nanmedian(temp,axis=1)
        data2.append(temp)
        
    data2=np.array(data2)
    np.save(fname[:-4]+'_centres.npy',data2)
    print (data2.shape)

(4, 89988, 2)
(4, 89985, 2)
(4, 89987, 2)
(4, 89988, 2)
(4, 89987, 2)
(4, 89987, 2)
(4, 89988, 2)
(4, 89989, 2)
(4, 89976, 2)
(4, 89988, 2)
